## 准备数据

In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data() #加载数据
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

## Demo numpy based auto differentiation

In [8]:
import numpy as np

class Matmul:   #这一步是神经元计算
    def __init__(self):
        self.mem = {}
        
    def forward(self, x, W):  #前馈过程
        h = np.matmul(x, W) 
        self.mem={'x': x, 'W':W}
        return h
    
    def backward(self, grad_y):   #反馈过程
        '''
        x: shape(N, d)
        w: shape(d, d')
        grad_y: shape(N, d')
        '''
        x = self.mem['x']
        W = self.mem['W']
        
        ####################
        grad_x = np.matmul(grad_y, W.T) # shape(N, b)
        grad_W = np.matmul(x.T, grad_y)
        ####################
        
        return grad_x, grad_W


class Relu:   #Relu激活函数
    def __init__(self):
        self.mem = {}
        
    def forward(self, x):
        self.mem['x']=x
        return np.where(x > 0, x, np.zeros_like(x))
    
    def backward(self, grad_y):
        '''
        grad_y: same shape as x
        '''
        ####################
        '''计算relu 激活函数对应的梯度'''
        x = self.mem['x']
        ####################
        return (x > 0).astype(np.float32) * grad_y
    


class Softmax:   #softmax函数
    '''
    softmax over last dimention
    '''
    def __init__(self):
        self.epsilon = 1e-12
        self.mem = {}
        
    def forward(self, x):
        '''
        x: shape(N, c)
        '''
        x_exp = np.exp(x)
        partition = np.sum(x_exp, axis=1, keepdims=True)
        out = x_exp/(partition+self.epsilon)
        
        self.mem['out'] = out
        self.mem['x_exp'] = x_exp
        return out
    
    def backward(self, grad_y):
        '''
        grad_y: same shape as x
        '''
        s = self.mem['out']
        sisj = np.matmul(np.expand_dims(s,axis=2), np.expand_dims(s, axis=1)) # (N, c, c)
        g_y_exp = np.expand_dims(grad_y, axis=1)
        tmp = np.matmul(g_y_exp, sisj) #(N, 1, c)
        tmp = np.squeeze(tmp, axis=1)
        tmp = -tmp+grad_y*s 
        return tmp
    
class Log:  #log函数
    '''
    softmax over last dimention
    '''
    def __init__(self):
        self.epsilon = 1e-12
        self.mem = {}
        
    def forward(self, x):
        '''
        x: shape(N, c)
        '''
        out = np.log(x+self.epsilon)
        
        self.mem['x'] = x
        return out
    
    def backward(self, grad_y):
        '''
        grad_y: same shape as x
        '''
        x = self.mem['x']
        
        return 1./(x+1e-12) * grad_y
    


## Gradient check

In [5]:
# import tensorflow as tf

# x = np.random.normal(size=[5, 6])
# W = np.random.normal(size=[6, 4])
# aa = Matmul()
# out = aa.forward(x, W) # shape(5, 4)
# grad = aa.backward(np.ones_like(out))
# print (grad)

# with tf.GradientTape() as tape:
#     x, W = tf.constant(x), tf.constant(W)
#     tape.watch(x)
#     y = tf.matmul(x, W)
#     loss = tf.reduce_sum(y)
#     grads = tape.gradient(loss, x)
#     print (grads)

# import tensorflow as tf

# x = np.random.normal(size=[5, 6])
# aa = Relu()
# out = aa.forward(x) # shape(5, 4)
# grad = aa.backward(np.ones_like(out))
# print (grad)

# with tf.GradientTape() as tape:
#     x= tf.constant(x)
#     tape.watch(x)
#     y = tf.nn.relu(x)
#     loss = tf.reduce_sum(y)
#     grads = tape.gradient(loss, x)
#     print (grads)

# import tensorflow as tf
# x = np.random.normal(size=[5, 6], scale=5.0, loc=1)
# label = np.zeros_like(x)
# label[0, 1]=1.
# label[1, 0]=1
# label[1, 1]=1
# label[2, 3]=1
# label[3, 5]=1
# label[4, 0]=1
# print(label)
# aa = Softmax()
# out = aa.forward(x) # shape(5, 6)
# grad = aa.backward(label)
# print (grad)

# with tf.GradientTape() as tape:
#     x= tf.constant(x)
#     tape.watch(x)
#     y = tf.nn.softmax(x)
#     loss = tf.reduce_sum(y*label)
#     grads = tape.gradient(loss, x)
#     print (grads)

# import tensorflow as tf

# x = np.random.normal(size=[5, 6])
# aa = Log()
# out = aa.forward(x) # shape(5, 4)
# grad = aa.backward(label)
# print (grad)

# with tf.GradientTape() as tape:
#     x= tf.constant(x)
#     tape.watch(x)
#     y = tf.math.log(x)
#     loss = tf.reduce_sum(y*label)
#     grads = tape.gradient(loss, x)
#     print (grads)

# Final Gradient Check

In [9]:
import tensorflow as tf   #这个和下面的模型重复，注释在下面模型上
x = np.random.normal(size=[5, 6])
label = np.zeros_like(x)
label[0, 1]=1.
label[1, 0]=1
label[2, 3]=1
label[3, 5]=1
label[4, 0]=1


W1 = np.random.normal(size=[6, 5])
W2 = np.random.normal(size=[5, 6])

mul_h1 = Matmul()
mul_h2 = Matmul()
relu = Relu()
softmax = Softmax()
log = Log()

h1 = mul_h1.forward(x, W1) # shape(5, 4)
h1_relu = relu.forward(h1)
h2 = mul_h2.forward(h1_relu, W2)
h2_soft = softmax.forward(h2)
h2_log = log.forward(h2_soft)


h2_log_grad = log.backward(label)
h2_soft_grad = softmax.backward(h2_log_grad)
h2_grad, W2_grad = mul_h2.backward(h2_soft_grad)
h1_relu_grad = relu.backward(h2_grad)
h1_grad, W1_grad = mul_h1.backward(h1_relu_grad)

print(h2_log_grad)
print('--'*20)
# print(W2_grad)

with tf.GradientTape() as tape:  #以梯度下降法运行下面步骤
    x, W1, W2, label = tf.constant(x), tf.constant(W1), tf.constant(W2), tf.constant(label)
    tape.watch(W1)
    tape.watch(W2)
    h1 = tf.matmul(x, W1)
    h1_relu = tf.nn.relu(h1)
    h2 = tf.matmul(h1_relu, W2)
    prob = tf.nn.softmax(h2)
    log_prob = tf.math.log(prob)
    loss = tf.reduce_sum(label * log_prob)
    grads = tape.gradient(loss, [prob])
    print (grads[0].numpy())

[[0.00000000e+00 2.27912732e+01 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [7.91989530e+05 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 3.06820307e+06
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 4.83500934e+00]
 [1.23855942e+04 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
----------------------------------------
[[0.00000000e+00 2.27912732e+01 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [7.91990157e+05 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 3.06821249e+06
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 4.83500934e+00]
 [1.23855944e+04 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]


## 建立模型

In [10]:
class myModel:  
    def __init__(self):   #初始化权重
        
        self.W1 = np.random.normal(size=[28*28+1, 100])
        self.W2 = np.random.normal(size=[100, 10])
        
        self.mul_h1 = Matmul()
        self.mul_h2 = Matmul()
        self.relu = Relu()  #各个函数
        self.softmax = Softmax()
        self.log = Log()
        
        
    def forward(self, x):   #前馈过程
        x = x.reshape(-1, 28*28)  #初始化x 
        bias = np.ones(shape=[x.shape[0], 1])#初始化x偏置
        x = np.concatenate([x, bias], axis=1)
        
        self.h1 = self.mul_h1.forward(x, self.W1) # shape(5, 4)  通过一层神经元
        self.h1_relu = self.relu.forward(self.h1)#通过Relu激活
        self.h2 = self.mul_h2.forward(self.h1_relu, self.W2)#通过第二个神经元
        self.h2_soft = self.softmax.forward(self.h2)# 通过softmax
        self.h2_log = self.log.forward(self.h2_soft)#通过log函数激活
            
    def backward(self, label):  #返向传播过程
        self.h2_log_grad = self.log.backward(-label)  #log的梯度
        self.h2_soft_grad = self.softmax.backward(self.h2_log_grad)#softmax的梯度
        self.h2_grad, self.W2_grad = self.mul_h2.backward(self.h2_soft_grad)#第二层的神经元的误差和W_delta
        self.h1_relu_grad = self.relu.backward(self.h2_grad)# RELU梯度
        self.h1_grad, self.W1_grad = self.mul_h1.backward(self.h1_relu_grad)#第一层的神经元的误差和W_delta
        
model = myModel()


## 计算 loss

In [11]:
def compute_loss(log_prob, labels):
     return np.mean(np.sum(-log_prob*labels, axis=1)) #损失函数
    

def compute_accuracy(log_prob, labels):
    predictions = np.argmax(log_prob, axis=1) #预测值最大池化
    truth = np.argmax(labels, axis=1)#真实值最大池化
    return np.mean(predictions==truth) # 计算差异平均值

def train_one_step(model, x, y):
    model.forward(x)  #运行前馈层
    model.backward(y)  #反向传播
    model.W1 -= 1e-5* model.W1_grad  #计算W1的变化
    model.W2 -= 1e-5* model.W2_grad  # 计算W2变化
    loss = compute_loss(model.h2_log, y)  # 把log后的预测值输入上面损失函数，构成交叉熵损失
    accuracy = compute_accuracy(model.h2_log, y)# 计算精确度
    return loss, accuracy

def test(model, x, y):#测试集重复上面步骤
    model.forward(x)  
    loss = compute_loss(model.h2_log, y)
    accuracy = compute_accuracy(model.h2_log, y)
    return loss, accuracy

## 实际训练

In [13]:
train_data, test_data = mnist_dataset()#生产数据集 
train_label = np.zeros(shape=[train_data[0].shape[0], 10])
test_label = np.zeros(shape=[test_data[0].shape[0], 10])
train_label[np.arange(train_data[0].shape[0]), np.array(train_data[1])] = 1.
test_label[np.arange(test_data[0].shape[0]), np.array(test_data[1])] = 1.

for epoch in range(100): #迭代100次
    loss, accuracy = train_one_step(model, train_data[0], train_label)
    print('epoch', epoch, ': loss', loss, '; accuracy', accuracy)
loss, accuracy = test(model, test_data[0], test_label)

print('test loss', loss, '; accuracy', accuracy)

epoch 0 : loss 3.7989240110146687 ; accuracy 0.7968833333333334
epoch 1 : loss 3.8410128795863603 ; accuracy 0.79565
epoch 2 : loss 3.8882392791876135 ; accuracy 0.7930166666666667
epoch 3 : loss 3.797132332780846 ; accuracy 0.7985666666666666
epoch 4 : loss 3.7400431378828185 ; accuracy 0.8003333333333333
epoch 5 : loss 3.559173073742702 ; accuracy 0.8094166666666667
epoch 6 : loss 3.4489878315937714 ; accuracy 0.8134333333333333
epoch 7 : loss 3.294548508311133 ; accuracy 0.82145
epoch 8 : loss 3.205648074190663 ; accuracy 0.8243
epoch 9 : loss 3.118361385729943 ; accuracy 0.8295333333333333
epoch 10 : loss 3.050042179749537 ; accuracy 0.8321166666666666
epoch 11 : loss 2.998864069233772 ; accuracy 0.8345833333333333
epoch 12 : loss 2.9535824568847118 ; accuracy 0.8363
epoch 13 : loss 2.9152380963347735 ; accuracy 0.8381333333333333
epoch 14 : loss 2.8813562696774917 ; accuracy 0.83965
epoch 15 : loss 2.8484371066140235 ; accuracy 0.8410833333333333
epoch 16 : loss 2.8207329421323504